In [1]:
# Import dependencies:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# Part 1: Data Exploration

In [2]:
# Read original dataset from train.csv and store it in a dataframe:

df = pd.read_csv("Data/train.csv")
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [ ]:
# Check which columns contain any missing values:

df.columns[df.isnull().any()]

In [ ]:
# Check the percentage of missing values in the columns that have missing values:

proportion_missing = df.isnull().sum()/len(df)
proportion_missing = proportion_missing[proportion_missing > 0]
proportion_missing.sort_values(inplace=True)
proportion_missing

In [ ]:
# Check the distribution of the sale prices (our target variable):

plt.hist(df.SalePrice, alpha=0.70, align="left")
plt.xlabel("House Price")
plt.ylabel("Number of Houses")
plt.xticks(rotation=90)
plt.title("Sale Price Distribution")
plt.show()

In [ ]:
# Find quartiles of the sale price distribution:

sale_price = df["SalePrice"]

quartiles = sale_price.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"Lower quartile: {lowerq}")
print(f"Upper quartile: {upperq}")
print(f"Interquartile range: {iqr}")
print(f"Median: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Create a boxplot of the sale price distribution:

fig1, ax1 = plt.subplots()
ax1.set_title("Sale Price Distribution")
ax1.set_ylabel("House Price")
ax1.boxplot(sale_price)
plt.show()

In [ ]:
# Separate data by categorical and numerical values:

numeric_data = df.select_dtypes(include=[np.number])
categorical_data = df.select_dtypes(exclude=[np.number])

print(f"Number of columns containing categorical data: {categorical_data.shape[1]}")
print(f"Number of columns containing numeric data: {numeric_data.shape[1]}")

In [ ]:
# For numeric data, create a correlation matrix that will help us visualize correlations between our features and target value:

corr = numeric_data.corr()
sns.heatmap(corr, cmap="BuPu")

In [ ]:
# Print correlation values of the features with the strongest positive correlations to the sale price:

print (corr['SalePrice'].sort_values(ascending=False)[:15], '\n')

In [ ]:
# Print correlation values of the features with the strongest negative correlations to the sale price:

print (corr['SalePrice'].sort_values(ascending=False)[-10:])

# Part 2: Data Preprocessing

In [3]:
# Drop the 5 columns that contain that contain the highest proportions of missing values, as discovered during data exploration:

df = df.drop(columns=["FireplaceQu", "Fence", "Alley", "MiscFeature", "PoolQC"])
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [4]:
# Drop rows with missing values:

df = df.dropna()
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [6]:
# Remove the "Id" column from the dataset:

df = df.drop(columns = ["Id"])
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,112,0,0,0,0,4,2010,WD,Normal,142125


In [7]:
# Use "get_dummies" to encode categorical variables into numeric values:

df = pd.get_dummies(df, prefix_sep='_', drop_first=True)
df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,1,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,1,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,1,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,0,1,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,0,1,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,0,1,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,0,0,0,1,0,0,0,1,0


In [8]:
# Store the cleaned dataframe as a csv:

df.to_csv("Data/cleaned_data.csv", index=False, header=True)

# Part 3: Determining Feature Importance

In [9]:
# Using the cleaned dataframe, isolate the target variable ("SalePrice") and remove it from the dataset:

X = df.drop(columns = ["SalePrice"])
Y = df["SalePrice"]

In [10]:
# Instantiate a Random Forest Regressor model and fit it to the dataset. Calculate the model's score:

rf = RandomForestRegressor()
rf = rf.fit(X, Y)
rf.score(X, Y)

0.9782032535637033

In [11]:
# Determine the importance of each feature to the target variable:

sorted_fi = sorted(zip(rf.feature_importances_, X.columns), reverse=True)
sorted_fi

[(0.6177051790260153, 'OverallQual'),
 (0.09422076751376987, 'GrLivArea'),
 (0.04611041033387128, '2ndFlrSF'),
 (0.026347246787632728, 'TotalBsmtSF'),
 (0.02486255311843789, 'BsmtFinSF1'),
 (0.020844083103896217, '1stFlrSF'),
 (0.013964150188502864, 'FullBath'),
 (0.012568409203472469, 'LotArea'),
 (0.010687124065128619, 'GarageCars'),
 (0.009893430231383359, 'GarageArea'),
 (0.0088333876624706, 'YearRemodAdd'),
 (0.008543872780221725, 'YearBuilt'),
 (0.007357082576685626, 'TotRmsAbvGrd'),
 (0.00608935402901386, 'BsmtUnfSF'),
 (0.0059002233032302625, 'OpenPorchSF'),
 (0.00579366321040909, 'LotFrontage'),
 (0.005124750551149838, 'GarageYrBlt'),
 (0.004708803687163731, 'WoodDeckSF'),
 (0.004485674214007726, 'MasVnrArea'),
 (0.0036504151879872713, 'OverallCond'),
 (0.0033050433328994395, 'BsmtQual_Gd'),
 (0.003230200355545917, 'MoSold'),
 (0.0031989292740488683, 'ExterQual_Gd'),
 (0.0028719267498969058, 'GarageType_Detchd'),
 (0.002522137536261171, 'Fireplaces'),
 (0.002268841129189609, '

In [15]:
# Only keep the features that have feature importance values above 0.01. These will be the features that we will use to train our models:

cols = []

for fi, column in sorted_fi: 
    if fi >= 0.01:
        cols.append(column)

cols

['OverallQual',
 'GrLivArea',
 '2ndFlrSF',
 'TotalBsmtSF',
 'BsmtFinSF1',
 '1stFlrSF',
 'FullBath',
 'LotArea',
 'GarageCars']